In [1]:
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
import numpy as np

In [19]:
trainSet = pd.read_excel('trainSetFrom_data_accident_final_17.10.17_17.10.24.xlsx')
testSet = pd.read_excel('testSetFrom_data_accident_final_17.10.17_17.10.24.xlsx')

In [20]:
trainSet.columns

Index(['index', '所辖乡镇', '行政区划', '事故发生时间', '星期', '路号', '受伤人数', '当事人总数',
       '抢救死亡人数Q', '机动车数量', '死亡人数', '直接财产损失', '米数', '轻伤人数', '重伤人数', '非机动车数量',
       '行人数量', '事故形态', '事故类型', '单车事故', '是否简易程序', '是否路外事故', '车辆间事故.1', '逃逸事故侦破',
       '事故认定原因', '事故认定原因分类小类', 'index.1'],
      dtype='object')

In [21]:
#将trainset与testset数字化
trainSet.drop(['index','index.1'],axis=1,inplace=True)
testSet.drop(['index','index.1'],axis=1,inplace=True)
trainSet.drop(['星期'],axis=1,inplace=True)
testSet.drop(['星期'],axis=1,inplace=True)



columnsToNum = list(trainSet.columns)
columnsToNum.remove('车辆间事故.1')
trainSet_toNum = pd.DataFrame()
for i in columnsToNum:
    trainSet_toNum[i] = preprocessing.LabelEncoder().fit_transform(trainSet[i])
trainSet_toNum.insert(value=list(trainSet['车辆间事故.1']),column='车辆间事故.1',loc=21)

columnsToNum = list(testSet.columns)
columnsToNum.remove('车辆间事故.1')
testSet_toNum = pd.DataFrame()
for i in columnsToNum:
    testSet_toNum[i] = preprocessing.LabelEncoder().fit_transform(testSet[i])
testSet_toNum.insert(value=list(testSet['车辆间事故.1']),column='车辆间事故.1',loc=21)

del trainSet,testSet

In [22]:
#统一训练集、测试集格式
x_train =np.array(trainSet_toNum.drop(['事故类型'],axis=1)) 
y_train = np.array(trainSet_toNum['事故类型'])
y = np.zeros((len(y_train),4))
for i in range(len(y_train)):
    y[i][y_train[i]] = 1
y_train = np.mat(y)
x_train = np.mat(x_train)
y_train = np.mat(y_train)


x_test =np.array(testSet_toNum.drop(['事故类型'],axis=1)) 
y_test = np.array(testSet_toNum['事故类型'])
y = np.zeros((len(y_test),4))
for i in range(len(y_test)):
    y[i][y_test[i]] = 1
y_test = np.mat(y)
x_test = np.mat(x_test)
y_test = np.mat(y_test)


del y,trainSet_toNum,testSet_toNum

In [23]:
print('训练集维度：%s'%str(x_train.shape))
print('训练集标签维度：%s'%str(y_train.shape))
print('测试集维度：%s'%str(x_test.shape))
print('测试集标签维度：%s'%str(y_test.shape))

训练集维度：(113344, 23)
训练集标签维度：(113344, 4)
测试集维度：(20001, 23)
测试集标签维度：(20001, 4)


In [26]:
def add_layer(inputs, in_size, out_size,n_layer, activation_function=None,):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s'%n_layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]),name='W'+str(n_layer))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1,name='b'+str(n_layer))
    Wx_plus_b = tf.matmul(inputs, Weights) + biases

    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b,)
    return outputs
     
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result


In [28]:
xs = tf.placeholder(tf.float32, [None, x_train.shape[1]],name='x_input') # 
ys = tf.placeholder(tf.float32, [None, y_train.shape[1]],name='y_input')

l1 = add_layer(xs, x_train.shape[1], 18,n_layer=1,  activation_function=tf.nn.tanh)
l2 = add_layer(l1, 18, 10,n_layer=2,  activation_function=tf.nn.tanh)
prediction = add_layer(l2,10,y_train.shape[1],n_layer=3,activation_function=tf.nn.softmax)
# the error between prediction and real data
loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(0.2).minimize(loss)

In [29]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(10):
    sess.run(train_step, feed_dict={xs: x_train, ys: y_train})
#     if i % 10 == 0:
    result = sess.run(train_step, feed_dict={xs: x_train, ys: y_train})
    print(compute_accuracy(x_train, y_train))
#         writer.add_summary(result,i)

0.986237
0.986237
0.986237
0.986237
0.986237
0.986237
0.986237
0.986237
0.986237
0.986237


In [30]:
print(compute_accuracy(x_train, y_train))
print(compute_accuracy(x_test, y_test))

0.986237
0.986251


In [2]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=10000,random_state=0,n_jobs=-1)
forest.fit(x_train,y_train)

In [52]:
corr_pre_train = np.equal(np.argmax(forest.predict(x_train),axis=1),\
                    np.argmax(y_train[0:3],axis=1,).reshape( (1,len(np.argmax(y_train,axis=1,))) )\
                   )
acc_train = np.mean(corr_pre_train)

corr_pre_test = np.equal(np.argmax(forest.predict(x_test),axis=1),\
                    np.argmax(y_test[0:3],axis=1,).reshape( (1,len(np.argmax(y_test,axis=1,))) )\
                   )
acc_test = np.mean(corr_pre_test)

print('训练集准确率：%s'%acc_train)
print('测试集准确率：%s'%acc_test)

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
1.0
